In [1]:
import random
import itertools
import numpy as np
from operator import eq, neg
from sortedcontainers import SortedSet
import time

In [2]:
class NaryCSP:
    """
    A nary-CSP consists of:
    domains     : a dictionary that maps each variable to its domain
    constraints : a list of constraints
    variables   : a set of variables
    var_to_const: a variable to set of constraints dictionary
    """

    def __init__(self, domains, constraints):
        """Domains is a variable:domain dictionary
        constraints is a list of constraints
        """
        self.variables = set(domains)
        self.domains = domains
        self.constraints = constraints
        self.curr_domains = None
        self.nassigns = 0
        self.ntries = 0
        self.nprunings = 0
        self.var_to_const = {var: set() for var in self.variables}
        for con in constraints:
            for var in con.scope:
                self.var_to_const[var].add(con)

    def __str__(self):
        """String representation of CSP"""
        return str(self.domains)

    def display(self, assignment=None):
        """More detailed string representation of CSP"""
        if assignment is None:
            assignment = {}
        print(assignment)
        
    def choices(self, var):
        """Return all values for var that aren't currently ruled out."""
        return (self.curr_domains or self.domains)[var]
    
    def nconflicts(self, var, val, assignment):
        """Return the number of conflicts var=val has with other variables."""
        count = 0 
        assignment[var] = val
        for con in self.var_to_const[var]:
            if (set(con.scope)).issubset(set(assignment)) and not con.holds(assignment):
                count = count + 1
        del assignment[var]
        return count
    
    def find_neighbors(self, var):
        """Return the list of neighbors of a given variable."""
        neighbors = []
        for con in self.var_to_const[var]:
            neighbors.extend(list(con.scope))
        neighbbors = list(set(neighbors))
        neighbbors.remove(var)
    
        return neighbbors

    
    def assign(self, var, val, assignment):
        """Add {var: val} to assignment; Discard the old value if any."""
        assignment[var] = val
        self.nassigns += 1
        
    def suppose(self, var, value):
        """Start accumulating inferences from assuming var=value."""
        self.support_pruning()
        removals = [(var, a) for a in self.curr_domains[var] if a != value]
        self.curr_domains[var] = [value]
        return removals
    
    def restore(self, removals):
        """Undo a supposition and all inferences from it."""
        for B, b in removals:
            self.curr_domains[B].append(b)

    def consistent(self, assignment):
        """assignment is a variable:value dictionary
        returns True if all of the constraints that can be evaluated
                        evaluate to True given assignment.
        """
        return all(con.holds(assignment)
                   for con in self.constraints
                   if all(v in assignment for v in con.scope))
    
    def prune(self, var, value, removals):
        """Rule out var=value."""
        self.curr_domains[var].remove(value)
        if removals is not None:
            removals.append((var, value))
    
    def support_pruning(self):
        """Make sure we can prune values from domains. (We want to pay
        for this only if we use it.)"""
        if self.curr_domains is None:
            self.curr_domains = {v: list(self.domains[v]) for v in self.variables}
            
    def unassign(self, var, assignment):
        """Remove {var: val} from assignment.
        DO NOT call this if you are changing a variable to a new value;
        just call assign for that."""
        if var in assignment:
            del assignment[var]
            
    def goal_test(self, state):
        """The goal is to assign all variables, with all constraints satisfied."""
        assignment = dict(state)
        return (len(assignment) == len(self.variables)
                and all(self.nconflicts(variables, assignment[variables], assignment) == 0
                        for variables in self.variables))
    
    # This is for min_conflicts search
    def conflicted_vars(self, current):
        """Return a list of variables in current assignment that are in conflict"""
        return [var for var in self.variables
                if self.nconflicts(var, current[var], current) > 0]

In [3]:
class Constraint:
    """
    A Constraint consists of:
    scope    : a tuple of variables
    condition: a function that can applied to a tuple of values
    for the variables.
    """

    def __init__(self, scope, condition):
        self.scope = scope
        self.condition = condition

    def __repr__(self):
        return self.condition.__name__ + str(self.scope) + str(self.condition)
    
    def name(self):
        return self.condition.__name__

    def holds(self, assignment):
        """Returns the value of Constraint con evaluated in assignment.

        precondition: all variables are assigned in assignment
        """
        return self.condition(*tuple(assignment[v] for v in self.scope))

In [4]:
def find_all_diff_constraints(var_list):
    cons = []
    for index, var1 in enumerate(var_list):
        for var2 in var_list[index+1:]:
            cons.append(Constraint((var1, var2), lambda a, b: a != b))
    return cons

def first_unassigned_variable(assignment, csp):
    """The default variable order."""
    return first([var for var in csp.variables if var not in assignment])

def unordered_domain_values(var, assignment, csp):
    """The default value order."""
    return csp.choices(var)

def no_inference(csp, var, value, assignment, removals):
    return True

def first(iterable, default=None):
    """Return the first element of an iterable; or default."""
    return next(iter(iterable), default)

def count(seq):
    """Count the number of items in sequence that are interpreted as true."""
    return sum(map(bool, seq))

identity = lambda x: x

def shuffled(iterable):
    """Randomly shuffle a copy of iterable."""
    items = list(iterable)
    random.shuffle(items)
    return items

def argmin_random_tie(seq, key=identity):
    """Return a minimum element of seq; break ties at random."""
    return min(shuffled(seq), key=key)

def mrv(assignment, csp):
    """Minimum-remaining-values heuristic."""
    return argmin_random_tie([v for v in csp.variables if v not in assignment],
                             key=lambda var: num_legal_values(csp, var, assignment))


def num_legal_values(csp, var, assignment):
    if csp.curr_domains:
        return len(csp.curr_domains[var])
    else:
        return count(csp.nconflicts(var, val, assignment) == 0 for val in csp.domains[var])

    
def lcv(var, assignment, csp):
    """Least-constraining-values heuristic."""
    return sorted(csp.choices(var), key=lambda val: csp.nconflicts(var, val, assignment))


def forward_checking(csp, var, value, assignment, removals):
    """Prune neighbor values inconsistent with var=value."""
    
    csp.support_pruning()
    for con in csp.var_to_const[var]:
        neighbors = list(con.scope)
        neighbors.remove(var)
        unassigned_neighbors = []
        for neighbor in neighbors:
            if neighbor not in assignment.keys():
                unassigned_neighbors.append(neighbor)
        
        if len(unassigned_neighbors) == 1:
            csp.ntries = csp.ntries + 1
            unassigned_neighbor = unassigned_neighbors[0]
            for domain_val in csp.curr_domains[unassigned_neighbor]:
                assignment[unassigned_neighbor] = domain_val
                if not con.holds(assignment):
                    csp.prune(unassigned_neighbor, domain_val, removals)
                del assignment[unassigned_neighbor]
            
            if len(csp.curr_domains[unassigned_neighbor]) == 0:
                csp.nprunnings = csp.nprunnings + 1
                return False
        
        return True


def dom_j_up(csp, queue):
    return SortedSet(queue, key=lambda t: neg(len(csp.curr_domains[t[1]])))


def AC3(csp, assignment, queue=None, removals=None, arc_heuristic=dom_j_up):
    """[Figure 6.3]"""
    if queue is None:
        queue = {(Xi, Xk) for Xi in csp.variables for Xk in csp.find_neighbors(Xi)}
    csp.support_pruning()
    queue = arc_heuristic(csp, queue)
    checks = 0
    while queue:
        (Xi, Xj) = queue.pop()
        revised, checks = revise(csp, Xi, Xj, assignment, removals, checks)
        if revised:
            if not csp.curr_domains[Xi]:
                return False, checks  # CSP is inconsistent
            for Xk in csp.find_neighbors(Xi):
                if Xk != Xj:
                    queue.add((Xk, Xi))
    return True, checks  # CSP is satisfiable


def revise(csp, Xi, Xj, assignment, removals, checks=0):
    """Return true if we remove a value."""
    revised = False
    for con in csp.var_to_const[Xj]:
        neighbors = list(con.scope)
        if Xi in neighbors:
            if Xj in neighbors:
                neighbors.remove(Xj)
                neighbors.remove(Xi)
                unassigned_neighbors = []
                for neighbor in neighbors:
                    if neighbor not in assignment.keys():
                        unassigned_neighbors.append(neighbor)
        
                if len(unassigned_neighbors) == 0:
                    csp.ntries = csp.ntries + 1
                    for x in csp.curr_domains[Xi]:
                        conflict = True
                        assignment[Xi] = x
                        for y in csp.curr_domains[Xj]:
                            assignment[Xj] = y
                            if con.holds(assignment):
                                conflict = False
                            checks = checks + 1
                            if not conflict:
                                break
                        if conflict:
                            csp.prune(Xi, x, removals)
                            csp.nprunings = csp.nprunings + 1
                            revised = True
        return revised, checks


def mac(csp, var, value, assignment, removals, constraint_propagation=AC3):
    """Maintain arc consistency."""
    return constraint_propagation(csp, assignment, {(X, var) for X in csp.find_neighbors(var)}, removals)

In [5]:
def backtracking_search(csp, select_unassigned_variable=first_unassigned_variable,
                        order_domain_values=unordered_domain_values, inference=no_inference):

    def backtrack(assignment):
        if len(assignment) == len(csp.variables):
            return assignment
        
        var = select_unassigned_variable(assignment, csp)
        for value in order_domain_values(var, assignment, csp):
            if 0 == csp.nconflicts(var, value, assignment):
                csp.assign(var, value, assignment)
                removals = csp.suppose(var, value)
                if inference(csp, var, value, assignment, removals):
                    result = backtrack(assignment)
                    if result is not None:
                        return result
                csp.restore(removals)
        csp.unassign(var, assignment)
        return None

    result = backtrack({})
    assert result is None or csp.goal_test(result)
    return result

In [7]:
# T W O + T W O = F O U R
# 7 6 5 + 7 6 5 = 1 5 3 0
# 9 2 8 + 9 2 8 = 1 8 5 6
# 8 6 7 + 8 6 7 = 1 7 3 4
# 7 6 5 + 7 6 5 = 1 5 3 0
# 7 3 4 + 7 3 4 = 1 4 6 8
# 9 2 8 + 9 2 8 = 1 8 5 6


domains = {'T': set(range(1, 10)), 'F': set(range(1, 10)), 'W': set(range(0, 10)),
           'O': set(range(0, 10)), 'U': set(range(0, 10)), 'R': set(range(0, 10)),
           'C1': set(range(0, 2)), 'C2': set(range(0, 2)), 'C3': set(range(0, 2))}

constraints = [Constraint(('O', 'R', 'C1'), lambda o, r, c1: o + o == r + 10 * c1),
               Constraint(('W', 'U', 'C1', 'C2'), lambda w, u, c1, c2: c1 + w + w == u + 10 * c2),
               Constraint(('T', 'O', 'C2', 'C3'), lambda t, o, c2, c3: c2 + t + t == o + 10 * c3),
               Constraint(('F', 'C3'), lambda f, c3: f == c3)]

constraints.extend(find_all_diff_constraints(['T', 'F', 'W', 'O', 'U', 'R']))

two_two_four = NaryCSP(domains, constraints)

In [8]:
two_two_four.constraints

[<lambda>('O', 'R', 'C1')<function <lambda> at 0x7f941c636cb0>,
 <lambda>('W', 'U', 'C1', 'C2')<function <lambda> at 0x7f941c636d40>,
 <lambda>('T', 'O', 'C2', 'C3')<function <lambda> at 0x7f941c636dd0>,
 <lambda>('F', 'C3')<function <lambda> at 0x7f941c636e60>,
 <lambda>('T', 'F')<function find_all_diff_constraints.<locals>.<lambda> at 0x7f941c636ef0>,
 <lambda>('T', 'W')<function find_all_diff_constraints.<locals>.<lambda> at 0x7f941c636f80>,
 <lambda>('T', 'O')<function find_all_diff_constraints.<locals>.<lambda> at 0x7f941c645050>,
 <lambda>('T', 'U')<function find_all_diff_constraints.<locals>.<lambda> at 0x7f941c6450e0>,
 <lambda>('T', 'R')<function find_all_diff_constraints.<locals>.<lambda> at 0x7f941c645170>,
 <lambda>('F', 'W')<function find_all_diff_constraints.<locals>.<lambda> at 0x7f941c645200>,
 <lambda>('F', 'O')<function find_all_diff_constraints.<locals>.<lambda> at 0x7f941c645290>,
 <lambda>('F', 'U')<function find_all_diff_constraints.<locals>.<lambda> at 0x7f941c64

In [9]:
result = backtracking_search(two_two_four)
print("BT:", result)

result = backtracking_search(two_two_four, 
                             select_unassigned_variable=mrv)
print("BT + MRV: ", result)

result = backtracking_search(two_two_four, select_unassigned_variable=mrv, 
                             order_domain_values=lcv)
print("BT + MRV + LCV:", result)

result = backtracking_search(two_two_four, 
                             select_unassigned_variable=mrv, 
                             order_domain_values=lcv, 
                             inference=forward_checking)
print("BT + MRV + LCV + FC:", result)

result = backtracking_search(two_two_four, 
                             select_unassigned_variable=mrv, 
                             order_domain_values=lcv, 
                             inference=mac)
print("BT + MRV + LCV + MAC:", result)

BT: {'O': 4, 'C3': 1, 'C2': 0, 'F': 1, 'R': 8, 'W': 3, 'U': 6, 'C1': 0, 'T': 7}
BT + MRV:  {'C1': 0, 'C3': 1, 'R': 8, 'C2': 0, 'O': 4, 'F': 1, 'U': 6, 'W': 3, 'T': 7}
BT + MRV + LCV: {'C3': 1, 'R': 8, 'F': 1, 'C1': 0, 'C2': 0, 'W': 3, 'U': 6, 'T': 7, 'O': 4}
BT + MRV + LCV + FC: {'U': 6, 'C2': 0, 'O': 4, 'R': 8, 'T': 7, 'C3': 1, 'C1': 0, 'W': 3, 'F': 1}
BT + MRV + LCV + MAC: {'C1': 0, 'R': 8, 'O': 4, 'F': 1, 'W': 3, 'C2': 0, 'C3': 1, 'T': 7, 'U': 6}


In [10]:
# S E N D + M O R E = M O N E Y
# 9 5 6 7 + 1 0 8 5 = 1 0 6 5 2


domain_2 = {'S': set(range(1, 10)), 'M': set(range(1, 10)),
            'E': set(range(0, 10)), 'N': set(range(0, 10)), 'D': set(range(0, 10)),
            'O': set(range(0, 10)), 'R': set(range(0, 10)), 'Y': set(range(0, 10)),
            'C1': set(range(0, 2)), 'C2': set(range(0, 2)), 'C3': set(range(0, 2)),
            'C4': set(range(0, 2))}

constraints_2 = [Constraint(('D', 'E', 'Y', 'C1'), lambda d, e, y, c1: d + e == y + 10 * c1),
                 Constraint(('N', 'R', 'E', 'C1', 'C2'), lambda n, r, e, c1, c2: c1 + n + r == e + 10 * c2),
                 Constraint(('E', 'O', 'N', 'C2', 'C3'), lambda e, o, n, c2, c3: c2 + e + o == n + 10 * c3),
                 Constraint(('S', 'M', 'O', 'C3', 'C4'), lambda s, m, o, c3, c4: c3 + s + m == o + 10 * c4),
                 Constraint(('M', 'C4'), lambda m, c4: m == c4)]

constraints_2.extend(find_all_diff_constraints(['S', 'E', 'N', 'D', 'M', 'O', 'Y', 'R']))

send_more_money = NaryCSP(domain_2, constraints_2)

In [11]:
result = backtracking_search(send_more_money)
print("BT:", result)

result = backtracking_search(send_more_money, 
                             select_unassigned_variable=mrv)
print("BT + MRV: ", result)

result = backtracking_search(send_more_money, select_unassigned_variable=mrv, 
                             order_domain_values=lcv)
print("BT + MRV + LCV:", result)

result = backtracking_search(send_more_money, 
                             select_unassigned_variable=mrv, 
                             order_domain_values=lcv, 
                             inference=forward_checking)
print("BT + MRV + LCV + FC:", result)

result = backtracking_search(send_more_money, 
                             select_unassigned_variable=mrv, 
                             order_domain_values=lcv, 
                             inference=mac)
print("BT + MRV + LCV + MAC:", result)

BT: {'E': 5, 'O': 0, 'C3': 0, 'C4': 1, 'S': 9, 'C2': 1, 'D': 7, 'M': 1, 'R': 8, 'Y': 2, 'N': 6, 'C1': 1}
BT + MRV:  {'S': 9, 'E': 5, 'D': 7, 'M': 1, 'C1': 1, 'C3': 0, 'C2': 1, 'Y': 2, 'C4': 1, 'R': 8, 'O': 0, 'N': 6}
BT + MRV + LCV: {'O': 0, 'M': 1, 'N': 6, 'C1': 1, 'D': 7, 'E': 5, 'C3': 0, 'R': 8, 'C4': 1, 'S': 9, 'Y': 2, 'C2': 1}
BT + MRV + LCV + FC: {'C3': 0, 'C2': 1, 'E': 5, 'Y': 2, 'C1': 1, 'N': 6, 'S': 9, 'D': 7, 'R': 8, 'O': 0, 'M': 1, 'C4': 1}
BT + MRV + LCV + MAC: {'R': 8, 'N': 6, 'E': 5, 'D': 7, 'S': 9, 'C1': 1, 'C2': 1, 'C3': 0, 'O': 0, 'C4': 1, 'M': 1, 'Y': 2}


In [12]:
# C R O S S + R O A D S = D A N G E R
# 9 6 2 3 3 + 6 2 5 1 3 = 1 5 8 7 4 6

domain_3 = {'C': set(range(1, 10)), 'R': set(range(1, 10)), 'D': set(range(1, 10)),
            'O': set(range(0, 10)), 'S': set(range(0, 10)), 'A': set(range(0, 10)),
            'N': set(range(0, 10)), 'G': set(range(0, 10)), 'E': set(range(0, 10)),
            'C1': set(range(0, 2)), 'C2': set(range(0, 2)), 'C3': set(range(0, 2)),
            'C4': set(range(0, 2)), 'C5': set(range(0, 2))}

constraints_3 = [Constraint(('S', 'R', 'C1'), lambda s, r, c1: s + s == r + 10 * c1),
                 Constraint(('S', 'D', 'E', 'C1', 'C2'), lambda s, d, e, c1, c2: c1 + s + d == e + 10 * c2),
                 Constraint(('O', 'A', 'G', 'C2', 'C3'), lambda o, a, g, c2, c3: c2 + o + a == g + 10 * c3),
                 Constraint(('R', 'O', 'N', 'C3', 'C4'), lambda r, o, n, c3, c4: c3 + r + o == n + 10 * c4),
                 Constraint(('C', 'R', 'A', 'C4', 'C5'), lambda c, r, a, c4, c5: c4 + c + r == a + 10 * c5),
                 Constraint(('D', 'C5'), lambda d, c5: d == c5)]

constraints_3.extend(find_all_diff_constraints(['C', 'R', 'O', 'S', 'A', 'D', 'N', 'G', 'E']))

cross_roads_danger = NaryCSP(domain_3, constraints_3)

In [13]:
result = backtracking_search(cross_roads_danger)
print("BT:", result)

result = backtracking_search(cross_roads_danger, 
                             select_unassigned_variable=mrv)
print("BT + MRV: ", result)

result = backtracking_search(cross_roads_danger, select_unassigned_variable=mrv, 
                             order_domain_values=lcv)
print("BT + MRV + LCV:", result)

result = backtracking_search(cross_roads_danger, 
                             select_unassigned_variable=mrv, 
                             order_domain_values=lcv, 
                             inference=forward_checking)
print("BT + MRV + LCV + FC:", result)

result = backtracking_search(cross_roads_danger, 
                             select_unassigned_variable=mrv, 
                             order_domain_values=lcv, 
                             inference=mac)
print("BT + MRV + LCV + MAC:", result)

BT: {'E': 4, 'O': 2, 'C3': 0, 'C4': 0, 'S': 3, 'C2': 0, 'C5': 1, 'C1': 0, 'D': 1, 'G': 7, 'R': 6, 'C': 9, 'N': 8, 'A': 5}
BT + MRV:  {'O': 2, 'C3': 0, 'C2': 0, 'C1': 0, 'D': 1, 'E': 4, 'S': 3, 'N': 8, 'C': 9, 'G': 7, 'C4': 0, 'A': 5, 'C5': 1, 'R': 6}
BT + MRV + LCV: {'D': 1, 'C1': 0, 'C': 9, 'C4': 0, 'O': 2, 'G': 7, 'R': 6, 'E': 4, 'S': 3, 'C2': 0, 'C3': 0, 'C5': 1, 'N': 8, 'A': 5}
BT + MRV + LCV + FC: {'C2': 0, 'O': 2, 'S': 3, 'C5': 1, 'C3': 0, 'E': 4, 'A': 5, 'D': 1, 'C4': 0, 'G': 7, 'C1': 0, 'R': 6, 'C': 9, 'N': 8}
BT + MRV + LCV + MAC: {'C4': 0, 'C2': 0, 'S': 3, 'C5': 1, 'O': 2, 'R': 6, 'N': 8, 'A': 5, 'D': 1, 'C3': 0, 'E': 4, 'C1': 0, 'C': 9, 'G': 7}


In [14]:
# B A R R E L + B R O O M S = S H O V E L S
# 8 9 3 3 6 0 + 8 3 2 2 4 1 = 1 7 2 5 6 0 1

domain_4 = {'B': set(range(1, 10)), 'S': set(range(1, 10)), 'A': set(range(0, 10)),
            'R': set(range(0, 10)), 'E': set(range(0, 10)), 'L': set(range(0, 10)),
            'O': set(range(0, 10)), 'M': set(range(0, 10)), 'H': set(range(0, 10)), 'V': set(range(0, 10)),
            'C1': set(range(0, 2)), 'C2': set(range(0, 2)), 'C3': set(range(0, 2)),
            'C4': set(range(0, 2)), 'C5': set(range(0, 2)), 'C6': set(range(0, 2))}

constraints_4 = [Constraint(('L', 'S', 'C1'), lambda l, s, c1: l + s == s + 10 * c1),
                 Constraint(('E', 'M', 'L', 'C1', 'C2'), lambda e, m, l, c1, c2: c1 + e + m == l + 10 * c2),
                 Constraint(('R', 'O', 'E', 'C2', 'C3'), lambda r, o, e, c2, c3: c2 + r + o == e + 10 * c3),
                 Constraint(('R', 'O', 'V', 'C3', 'C4'), lambda r, o, v, c3, c4: c3 + r + o == v + 10 * c4),
                 Constraint(('A', 'R', 'O', 'C4', 'C5'), lambda a, r, o, c4, c5: c4 + a + r == o + 10 * c5),
                 Constraint(('B', 'H', 'C5', 'C6'), lambda b, h, c5, c6: c5 + b + b == h + 10 * c6),
                 Constraint(('S', 'C6'), lambda s, c6: s == c6)]

constraints_4.extend(find_all_diff_constraints(['B', 'A', 'R', 'E', 'L', 'O', 'M', 'S', 'H', 'V']))

barrel_brooms_shovels = NaryCSP(domain_4, constraints_4)

In [15]:
result = backtracking_search(barrel_brooms_shovels)
print("BT:", result)

result = backtracking_search(barrel_brooms_shovels, 
                             select_unassigned_variable=mrv)
print("BT + MRV: ", result)

result = backtracking_search(barrel_brooms_shovels, select_unassigned_variable=mrv, 
                             order_domain_values=lcv)
print("BT + MRV + LCV:", result)

result = backtracking_search(barrel_brooms_shovels, 
                             select_unassigned_variable=mrv, 
                             order_domain_values=lcv, 
                             inference=forward_checking)
print("BT + MRV + LCV + FC:", result)

result = backtracking_search(barrel_brooms_shovels, 
                             select_unassigned_variable=mrv, 
                             order_domain_values=lcv, 
                             inference=mac)
print("BT + MRV + LCV + MAC:", result)

BT: {'E': 6, 'C6': 1, 'O': 2, 'C3': 0, 'S': 1, 'C2': 1, 'C5': 1, 'V': 5, 'B': 8, 'R': 3, 'M': 4, 'L': 0, 'H': 7, 'C4': 0, 'C1': 0, 'A': 9}
BT + MRV:  {'L': 0, 'C4': 0, 'M': 4, 'R': 3, 'C6': 1, 'B': 8, 'C1': 0, 'C2': 1, 'A': 9, 'O': 2, 'C3': 0, 'H': 7, 'C5': 1, 'S': 1, 'E': 6, 'V': 5}
BT + MRV + LCV: {'C6': 1, 'L': 0, 'M': 4, 'S': 1, 'C4': 0, 'C3': 0, 'C2': 1, 'H': 7, 'R': 3, 'E': 6, 'C5': 1, 'V': 5, 'C1': 0, 'O': 2, 'A': 9, 'B': 8}
BT + MRV + LCV + FC: {'C4': 0, 'H': 7, 'L': 0, 'C1': 0, 'V': 5, 'C6': 1, 'S': 1, 'A': 9, 'C3': 0, 'M': 4, 'R': 3, 'C5': 1, 'O': 2, 'B': 8, 'E': 6, 'C2': 1}
BT + MRV + LCV + MAC: {'C6': 1, 'S': 1, 'V': 5, 'H': 7, 'L': 0, 'B': 8, 'M': 4, 'C1': 0, 'O': 2, 'A': 9, 'E': 6, 'C2': 1, 'R': 3, 'C5': 1, 'C4': 0, 'C3': 0}


In [16]:
def min_conflicts(csp, max_steps=10000):
    """Solve a CSP by stochastic Hill Climbing on the number of conflicts."""
    # Generate a complete assignment for all variables (probably with conflicts)
    current_assignment = {}
    for var in csp.variables:
        val = min_conflicts_value(csp, var, current_assignment)
        csp.assign(var, val, current_assignment)
        
    # Now repeatedly choose a random conflicted variable and change it
    for i in range(max_steps):
        conflicted = []
        for var in csp.variables:
            for con in csp.var_to_const[var]:
                if (set(con.scope)).issubset(set(current_assignment)) and not con.holds(current_assignment):
                    conflicted.append(var)
                    break
        if not conflicted:
            return current_assignment
        
        var = random.choice(conflicted)
        conflict_count = {}
        for val in csp.domains[var]:
            current_assignment[var] = val
            conflicted_vars = []
            for con in csp.var_to_const[var]:
                if (set(con.scope)).issubset(set(current_assignment)) and not con.holds(current_assignment):
                    conflicted_vars.extend(con.scope)
            
            if len(conflicted_vars) > 0:
                conflicted_vars = list(set(conflicted_vars))
                conflicted_vars.remove(var)
            
            conflict_count[val] = len(conflicted_vars)
        new_val = min(conflict_count.keys(), key=lambda x : conflict_count[x])
        current_assignment[var] = new_val
    return None

def min_conflicts_value(csp, var, current):
    """Return the value that will give var the least number of conflicts.
    If there is a tie, choose at random."""
    return argmin_random_tie(csp.domains[var], key=lambda val: csp.nconflicts(var, val, current))

In [17]:
result = min_conflicts(two_two_four)
print("TWO+TWO:", result)

result = min_conflicts(send_more_money)
print("SEND+MORE=MONEY:", result)

result = min_conflicts(cross_roads_danger)
print("CROSS+ROADS=DANGER:", result)

result = min_conflicts(barrel_brooms_shovels)
print("BARRELS+BROOMS=SHOVELS:", result)

TWO+TWO: None
SEND+MORE=MONEY: None
CROSS+ROADS=DANGER: None
BARRELS+BROOMS=SHOVELS: None


In [18]:
result = backtracking_search_new(cross_roads_danger)
print(result)

{'E': 4, 'O': 2, 'C3': 0, 'C4': 0, 'S': 3, 'C2': 0, 'C5': 1, 'C1': 0, 'D': 1, 'G': 7, 'R': 6, 'C': 9, 'N': 8, 'A': 5}


In [23]:
start = time.time()
random.seed(time.time()%100)
for i in range(0, 10):
    random.seed()
    result = backtracking_search_new(two_two_four)
    print(result)
end = time.time()
print(end - start)

{'O': 4, 'C3': 1, 'C2': 0, 'F': 1, 'R': 8, 'W': 3, 'U': 6, 'C1': 0, 'T': 7}
{'O': 4, 'C3': 1, 'C2': 0, 'F': 1, 'R': 8, 'W': 3, 'U': 6, 'C1': 0, 'T': 7}
{'O': 4, 'C3': 1, 'C2': 0, 'F': 1, 'R': 8, 'W': 3, 'U': 6, 'C1': 0, 'T': 7}
{'O': 4, 'C3': 1, 'C2': 0, 'F': 1, 'R': 8, 'W': 3, 'U': 6, 'C1': 0, 'T': 7}
{'O': 4, 'C3': 1, 'C2': 0, 'F': 1, 'R': 8, 'W': 3, 'U': 6, 'C1': 0, 'T': 7}
{'O': 4, 'C3': 1, 'C2': 0, 'F': 1, 'R': 8, 'W': 3, 'U': 6, 'C1': 0, 'T': 7}
{'O': 4, 'C3': 1, 'C2': 0, 'F': 1, 'R': 8, 'W': 3, 'U': 6, 'C1': 0, 'T': 7}
{'O': 4, 'C3': 1, 'C2': 0, 'F': 1, 'R': 8, 'W': 3, 'U': 6, 'C1': 0, 'T': 7}
{'O': 4, 'C3': 1, 'C2': 0, 'F': 1, 'R': 8, 'W': 3, 'U': 6, 'C1': 0, 'T': 7}
{'O': 4, 'C3': 1, 'C2': 0, 'F': 1, 'R': 8, 'W': 3, 'U': 6, 'C1': 0, 'T': 7}
0.007034778594970703


In [24]:
start = time.time()
random.seed(time.time()%100)
for i in range(0, 10):
    random.seed()
    result = backtracking_search_new(two_two_four, select_unassigned_variable=mrv)
    print(result)
end = time.time()
print(end - start)

{'U': 6, 'R': 8, 'T': 7, 'C3': 1, 'C1': 0, 'C2': 0, 'F': 1, 'O': 4, 'W': 3}
{'C3': 1, 'F': 1, 'T': 7, 'O': 4, 'R': 8, 'W': 3, 'C1': 0, 'C2': 0, 'U': 6}
{'C1': 0, 'F': 1, 'C2': 0, 'U': 6, 'O': 4, 'C3': 1, 'R': 8, 'W': 3, 'T': 7}
{'C2': 0, 'F': 1, 'T': 7, 'C3': 1, 'U': 6, 'W': 3, 'R': 8, 'C1': 0, 'O': 4}
{'C2': 0, 'O': 4, 'W': 3, 'C3': 1, 'R': 8, 'C1': 0, 'U': 6, 'F': 1, 'T': 7}
{'C2': 0, 'F': 1, 'R': 8, 'C3': 1, 'U': 6, 'T': 7, 'W': 3, 'C1': 0, 'O': 4}
{'C1': 0, 'R': 8, 'F': 1, 'W': 3, 'C3': 1, 'O': 4, 'U': 6, 'T': 7, 'C2': 0}
{'W': 3, 'F': 1, 'C2': 0, 'U': 6, 'C3': 1, 'C1': 0, 'R': 8, 'O': 4, 'T': 7}
{'U': 6, 'R': 8, 'O': 4, 'C3': 1, 'T': 7, 'W': 3, 'C1': 0, 'C2': 0, 'F': 1}
{'C2': 0, 'W': 3, 'T': 7, 'C3': 1, 'U': 6, 'C1': 0, 'O': 4, 'R': 8, 'F': 1}
0.007695198059082031


In [25]:
start = time.time()
random.seed(time.time()%100)
for i in range(0, 10):
    random.seed()
    result = backtracking_search_new(two_two_four, select_unassigned_variable=mrv, order_domain_values=lcv)
    print(result)
end = time.time()
print(end - start)

{'O': 4, 'R': 8, 'T': 7, 'C3': 1, 'F': 1, 'C1': 0, 'W': 3, 'U': 6, 'C2': 0}
{'O': 4, 'F': 1, 'T': 7, 'C2': 0, 'U': 6, 'R': 8, 'C1': 0, 'W': 3, 'C3': 1}
{'O': 4, 'C3': 1, 'R': 8, 'W': 3, 'C1': 0, 'T': 7, 'F': 1, 'U': 6, 'C2': 0}
{'F': 1, 'C3': 1, 'O': 4, 'C2': 0, 'T': 7, 'R': 8, 'W': 3, 'U': 6, 'C1': 0}
{'C1': 0, 'W': 3, 'R': 8, 'F': 1, 'C3': 1, 'C2': 0, 'U': 6, 'T': 7, 'O': 4}
{'C2': 0, 'O': 4, 'R': 8, 'C3': 1, 'T': 7, 'W': 3, 'U': 6, 'F': 1, 'C1': 0}
{'C1': 0, 'T': 7, 'R': 8, 'C2': 0, 'O': 4, 'W': 3, 'F': 1, 'C3': 1, 'U': 6}
{'O': 4, 'R': 8, 'C2': 0, 'U': 6, 'C3': 1, 'T': 7, 'F': 1, 'C1': 0, 'W': 3}
{'C1': 0, 'R': 8, 'W': 3, 'T': 7, 'U': 6, 'C3': 1, 'O': 4, 'F': 1, 'C2': 0}
{'C3': 1, 'U': 6, 'R': 8, 'F': 1, 'W': 3, 'T': 7, 'C2': 0, 'C1': 0, 'O': 4}
0.00568389892578125


In [26]:
start = time.time()
random.seed(time.time()%100)
for i in range(0, 10):
    random.seed()
    result = backtracking_search_new(two_two_four, 
                                     select_unassigned_variable=mrv, 
                                     order_domain_values=lcv, 
                                     inference=forward_checking)
    print(result)
end = time.time()
print(end - start)

{'R': 8, 'O': 4, 'C2': 0, 'T': 7, 'W': 3, 'C1': 0, 'F': 1, 'C3': 1, 'U': 6}
{'C3': 1, 'U': 6, 'R': 8, 'F': 1, 'W': 3, 'C2': 0, 'O': 4, 'T': 7, 'C1': 0}
{'C3': 1, 'C2': 0, 'O': 4, 'R': 8, 'W': 3, 'U': 6, 'C1': 0, 'T': 7, 'F': 1}
{'R': 8, 'C2': 0, 'T': 7, 'W': 3, 'U': 6, 'O': 4, 'F': 1, 'C3': 1, 'C1': 0}
{'T': 7, 'O': 4, 'R': 8, 'C1': 0, 'F': 1, 'U': 6, 'C2': 0, 'W': 3, 'C3': 1}
{'C3': 1, 'C1': 0, 'T': 7, 'O': 4, 'F': 1, 'C2': 0, 'W': 3, 'U': 6, 'R': 8}
{'F': 1, 'C3': 1, 'C2': 0, 'W': 3, 'O': 4, 'U': 6, 'R': 8, 'C1': 0, 'T': 7}
{'R': 8, 'U': 6, 'C1': 0, 'W': 3, 'O': 4, 'C2': 0, 'C3': 1, 'F': 1, 'T': 7}
{'O': 4, 'C1': 0, 'R': 8, 'W': 3, 'U': 6, 'F': 1, 'T': 7, 'C3': 1, 'C2': 0}
{'F': 1, 'C3': 1, 'O': 4, 'C2': 0, 'T': 7, 'U': 6, 'C1': 0, 'R': 8, 'W': 3}
0.007102251052856445


In [27]:
start = time.time()
random.seed(time.time()%100)
for i in range(0, 10):
    random.seed()
    result = backtracking_search_new(two_two_four, 
                                     select_unassigned_variable=mrv, 
                                     order_domain_values=lcv, 
                                     inference=mac)
    print(result)
end = time.time()
print(end - start)

{'T': 7, 'U': 6, 'C1': 0, 'C2': 0, 'F': 1, 'W': 3, 'C3': 1, 'O': 4, 'R': 8}
{'F': 1, 'W': 3, 'U': 6, 'R': 8, 'T': 7, 'O': 4, 'C3': 1, 'C2': 0, 'C1': 0}
{'O': 4, 'W': 3, 'U': 6, 'R': 8, 'T': 7, 'C3': 1, 'C2': 0, 'C1': 0, 'F': 1}
{'C1': 0, 'C2': 0, 'T': 7, 'U': 6, 'O': 4, 'W': 3, 'F': 1, 'C3': 1, 'R': 8}
{'O': 4, 'W': 3, 'R': 8, 'U': 6, 'T': 7, 'F': 1, 'C1': 0, 'C2': 0, 'C3': 1}
{'T': 7, 'U': 6, 'R': 8, 'O': 4, 'C1': 0, 'C3': 1, 'C2': 0, 'W': 3, 'F': 1}
{'C2': 0, 'F': 1, 'W': 3, 'O': 4, 'C3': 1, 'T': 7, 'C1': 0, 'U': 6, 'R': 8}
{'R': 8, 'O': 4, 'C1': 0, 'W': 3, 'F': 1, 'C2': 0, 'T': 7, 'U': 6, 'C3': 1}
{'T': 7, 'U': 6, 'C3': 1, 'C1': 0, 'R': 8, 'O': 4, 'W': 3, 'F': 1, 'C2': 0}
{'C3': 1, 'C1': 0, 'R': 8, 'O': 4, 'T': 7, 'U': 6, 'C2': 0, 'F': 1, 'W': 3}
0.01684284210205078


In [28]:
start = time.time()
random.seed(time.time()%100)
for i in range(0, 10):
    random.seed()
    result = backtracking_search_new(send_more_money)
    print(result)
end = time.time()
print(end - start)

{'E': 5, 'O': 0, 'C3': 0, 'C4': 1, 'S': 9, 'C2': 1, 'D': 7, 'M': 1, 'R': 8, 'Y': 2, 'N': 6, 'C1': 1}
{'E': 5, 'O': 0, 'C3': 0, 'C4': 1, 'S': 9, 'C2': 1, 'D': 7, 'M': 1, 'R': 8, 'Y': 2, 'N': 6, 'C1': 1}
{'E': 5, 'O': 0, 'C3': 0, 'C4': 1, 'S': 9, 'C2': 1, 'D': 7, 'M': 1, 'R': 8, 'Y': 2, 'N': 6, 'C1': 1}
{'E': 5, 'O': 0, 'C3': 0, 'C4': 1, 'S': 9, 'C2': 1, 'D': 7, 'M': 1, 'R': 8, 'Y': 2, 'N': 6, 'C1': 1}
{'E': 5, 'O': 0, 'C3': 0, 'C4': 1, 'S': 9, 'C2': 1, 'D': 7, 'M': 1, 'R': 8, 'Y': 2, 'N': 6, 'C1': 1}
{'E': 5, 'O': 0, 'C3': 0, 'C4': 1, 'S': 9, 'C2': 1, 'D': 7, 'M': 1, 'R': 8, 'Y': 2, 'N': 6, 'C1': 1}
{'E': 5, 'O': 0, 'C3': 0, 'C4': 1, 'S': 9, 'C2': 1, 'D': 7, 'M': 1, 'R': 8, 'Y': 2, 'N': 6, 'C1': 1}
{'E': 5, 'O': 0, 'C3': 0, 'C4': 1, 'S': 9, 'C2': 1, 'D': 7, 'M': 1, 'R': 8, 'Y': 2, 'N': 6, 'C1': 1}
{'E': 5, 'O': 0, 'C3': 0, 'C4': 1, 'S': 9, 'C2': 1, 'D': 7, 'M': 1, 'R': 8, 'Y': 2, 'N': 6, 'C1': 1}
{'E': 5, 'O': 0, 'C3': 0, 'C4': 1, 'S': 9, 'C2': 1, 'D': 7, 'M': 1, 'R': 8, 'Y': 2, 'N': 6,

In [29]:
start = time.time()
random.seed(time.time()%100)
for i in range(0, 10):
    random.seed()
    result = backtracking_search_new(send_more_money, select_unassigned_variable=mrv)
    print(result)
end = time.time()
print(end - start)

{'R': 8, 'Y': 2, 'S': 9, 'N': 6, 'O': 0, 'C3': 0, 'E': 5, 'C4': 1, 'D': 7, 'M': 1, 'C1': 1, 'C2': 1}
{'E': 5, 'R': 8, 'C1': 1, 'N': 6, 'S': 9, 'M': 1, 'C3': 0, 'C4': 1, 'D': 7, 'Y': 2, 'C2': 1, 'O': 0}
{'C1': 1, 'M': 1, 'R': 8, 'E': 5, 'C4': 1, 'C2': 1, 'N': 6, 'S': 9, 'D': 7, 'C3': 0, 'Y': 2, 'O': 0}
{'D': 7, 'C3': 0, 'N': 6, 'C4': 1, 'O': 0, 'Y': 2, 'C1': 1, 'S': 9, 'M': 1, 'C2': 1, 'E': 5, 'R': 8}
{'C4': 1, 'C1': 1, 'M': 1, 'S': 9, 'Y': 2, 'C3': 0, 'O': 0, 'E': 5, 'R': 8, 'D': 7, 'C2': 1, 'N': 6}
{'Y': 2, 'M': 1, 'S': 9, 'D': 7, 'R': 8, 'E': 5, 'C3': 0, 'O': 0, 'C4': 1, 'C1': 1, 'N': 6, 'C2': 1}
{'Y': 2, 'C4': 1, 'M': 1, 'S': 9, 'O': 0, 'C1': 1, 'E': 5, 'R': 8, 'N': 6, 'D': 7, 'C3': 0, 'C2': 1}
{'C1': 1, 'Y': 2, 'C3': 0, 'D': 7, 'S': 9, 'O': 0, 'M': 1, 'C2': 1, 'R': 8, 'N': 6, 'E': 5, 'C4': 1}
{'N': 6, 'Y': 2, 'C4': 1, 'C1': 1, 'E': 5, 'M': 1, 'C3': 0, 'D': 7, 'O': 0, 'S': 9, 'C2': 1, 'R': 8}
{'D': 7, 'O': 0, 'Y': 2, 'E': 5, 'S': 9, 'C3': 0, 'C1': 1, 'N': 6, 'C4': 1, 'M': 1, 'R': 8,

In [30]:
start = time.time()
random.seed(time.time()%100)
for i in range(0, 10):
    random.seed()
    result = backtracking_search_new(send_more_money, select_unassigned_variable=mrv, order_domain_values=lcv)
    print(result)
end = time.time()
print(end - start)

{'C1': 1, 'R': 8, 'S': 9, 'C4': 1, 'D': 7, 'Y': 2, 'E': 5, 'M': 1, 'O': 0, 'C3': 0, 'C2': 1, 'N': 6}
{'C1': 1, 'D': 7, 'M': 1, 'Y': 2, 'O': 0, 'C3': 0, 'C4': 1, 'E': 5, 'R': 8, 'S': 9, 'N': 6, 'C2': 1}
{'S': 9, 'Y': 2, 'N': 6, 'C2': 1, 'D': 7, 'M': 1, 'C1': 1, 'O': 0, 'C4': 1, 'R': 8, 'C3': 0, 'E': 5}
{'E': 5, 'N': 6, 'O': 0, 'C2': 1, 'S': 9, 'R': 8, 'C4': 1, 'D': 7, 'C1': 1, 'Y': 2, 'M': 1, 'C3': 0}
{'Y': 2, 'O': 0, 'M': 1, 'D': 7, 'C2': 1, 'C3': 0, 'E': 5, 'R': 8, 'N': 6, 'S': 9, 'C1': 1, 'C4': 1}
{'C1': 1, 'O': 0, 'Y': 2, 'C3': 0, 'E': 5, 'R': 8, 'C2': 1, 'N': 6, 'M': 1, 'S': 9, 'C4': 1, 'D': 7}
{'C2': 1, 'C4': 1, 'Y': 2, 'S': 9, 'D': 7, 'O': 0, 'C3': 0, 'M': 1, 'E': 5, 'C1': 1, 'N': 6, 'R': 8}
{'C3': 0, 'C1': 1, 'M': 1, 'R': 8, 'N': 6, 'Y': 2, 'D': 7, 'C4': 1, 'C2': 1, 'S': 9, 'E': 5, 'O': 0}
{'Y': 2, 'S': 9, 'C3': 0, 'E': 5, 'D': 7, 'M': 1, 'R': 8, 'O': 0, 'C2': 1, 'N': 6, 'C4': 1, 'C1': 1}
{'R': 8, 'C3': 0, 'C2': 1, 'D': 7, 'C4': 1, 'N': 6, 'M': 1, 'C1': 1, 'S': 9, 'E': 5, 'Y': 2

In [31]:
start = time.time()
random.seed(time.time()%100)
for i in range(0, 10):
    random.seed()
    result = backtracking_search_new(send_more_money, 
                                     select_unassigned_variable=mrv, 
                                     order_domain_values=lcv, 
                                     inference=forward_checking)
    print(result)
end = time.time()
print(end - start)

{'S': 9, 'R': 8, 'D': 7, 'E': 5, 'C2': 1, 'C1': 1, 'M': 1, 'O': 0, 'N': 6, 'C3': 0, 'C4': 1, 'Y': 2}
{'N': 6, 'C1': 1, 'S': 9, 'C2': 1, 'Y': 2, 'R': 8, 'M': 1, 'C4': 1, 'E': 5, 'C3': 0, 'O': 0, 'D': 7}
{'E': 5, 'R': 8, 'O': 0, 'S': 9, 'M': 1, 'D': 7, 'N': 6, 'Y': 2, 'C2': 1, 'C3': 0, 'C4': 1, 'C1': 1}
{'R': 8, 'M': 1, 'E': 5, 'N': 6, 'C4': 1, 'D': 7, 'S': 9, 'C1': 1, 'Y': 2, 'C3': 0, 'O': 0, 'C2': 1}
{'D': 7, 'C4': 1, 'C2': 1, 'O': 0, 'C3': 0, 'S': 9, 'M': 1, 'N': 6, 'Y': 2, 'E': 5, 'C1': 1, 'R': 8}
{'O': 0, 'E': 5, 'C2': 1, 'R': 8, 'M': 1, 'S': 9, 'D': 7, 'C1': 1, 'C4': 1, 'C3': 0, 'Y': 2, 'N': 6}
{'N': 6, 'C4': 1, 'D': 7, 'Y': 2, 'R': 8, 'M': 1, 'C3': 0, 'C2': 1, 'S': 9, 'O': 0, 'C1': 1, 'E': 5}
{'S': 9, 'C3': 0, 'E': 5, 'C1': 1, 'Y': 2, 'M': 1, 'C4': 1, 'D': 7, 'O': 0, 'C2': 1, 'N': 6, 'R': 8}
{'R': 8, 'D': 7, 'E': 5, 'Y': 2, 'C4': 1, 'C2': 1, 'N': 6, 'C1': 1, 'C3': 0, 'O': 0, 'M': 1, 'S': 9}
{'Y': 2, 'S': 9, 'C4': 1, 'C1': 1, 'N': 6, 'D': 7, 'E': 5, 'C3': 0, 'C2': 1, 'R': 8, 'O': 0

In [32]:
start = time.time()
random.seed(time.time()%100)
for i in range(0, 10):
    random.seed()
    result = backtracking_search_new(send_more_money, 
                                     select_unassigned_variable=mrv, 
                                     order_domain_values=lcv, 
                                     inference=mac)
    print(result)
end = time.time()
print(end - start)

{'O': 0, 'Y': 2, 'C2': 1, 'S': 9, 'N': 6, 'D': 7, 'E': 5, 'C3': 0, 'M': 1, 'C1': 1, 'R': 8, 'C4': 1}
{'N': 6, 'R': 8, 'Y': 2, 'M': 1, 'C4': 1, 'S': 9, 'C3': 0, 'C2': 1, 'D': 7, 'E': 5, 'C1': 1, 'O': 0}
{'E': 5, 'D': 7, 'C1': 1, 'C3': 0, 'Y': 2, 'M': 1, 'N': 6, 'R': 8, 'C4': 1, 'O': 0, 'C2': 1, 'S': 9}
{'C2': 1, 'E': 5, 'D': 7, 'N': 6, 'R': 8, 'M': 1, 'Y': 2, 'C3': 0, 'O': 0, 'C4': 1, 'S': 9, 'C1': 1}
{'M': 1, 'Y': 2, 'C1': 1, 'N': 6, 'R': 8, 'D': 7, 'E': 5, 'O': 0, 'C4': 1, 'C2': 1, 'C3': 0, 'S': 9}
{'D': 7, 'E': 5, 'Y': 2, 'M': 1, 'C3': 0, 'N': 6, 'R': 8, 'O': 0, 'S': 9, 'C2': 1, 'C4': 1, 'C1': 1}
{'S': 9, 'N': 6, 'R': 8, 'E': 5, 'D': 7, 'M': 1, 'Y': 2, 'O': 0, 'C2': 1, 'C3': 0, 'C4': 1, 'C1': 1}
{'D': 7, 'E': 5, 'Y': 2, 'M': 1, 'C3': 0, 'S': 9, 'N': 6, 'O': 0, 'R': 8, 'C4': 1, 'C2': 1, 'C1': 1}
{'C3': 0, 'C4': 1, 'M': 1, 'O': 0, 'Y': 2, 'R': 8, 'N': 6, 'C2': 1, 'E': 5, 'C1': 1, 'S': 9, 'D': 7}
{'M': 1, 'Y': 2, 'C4': 1, 'C3': 0, 'N': 6, 'R': 8, 'S': 9, 'C1': 1, 'C2': 1, 'D': 7, 'E': 5